In [11]:
# Bibliotecas
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [ ]:
# Carrega e instancia o cascade dentro da função Classificador Cascade do opencv
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Carrega e instancia o modelo classificador
model = load_model("modelo_01_expressoes.h5")
# Lista de Expressões que serão utilizadas para rótulo
expressoes = ["Raiva", "Nojo", "Medo", "Feliz", "Triste", "Surpreso", "Neutro"] 
# Pegando imagem em tempo real da Webcam
captura = cv2.VideoCapture(0)

while True:
    ret, frame = captura.read()
    
    # === Detectando faces ===
    # Converte video para escala de cinza para poder ajudar no processamento
    cinza = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    # Utiliza o detect multiscale para detectar as faces e retornará um tensor com vários pontos detectados 
    # (caso mais de uma face)
    faces = face_cascade.detectMultiScale(cinza,scaleFactor=1.2, minNeighbors=5, minSize=(100,100))
    for x,y,w,h in faces:
        frame = cv2.rectangle(frame,(x,y),(x+w,y+h+10),(255,50,50),2) # desenha retângulo ao redor da face
        frame = cv2.putText(frame, 'Face encontrada', (x,y-30),cv2.FONT_HERSHEY_SIMPLEX,1.1, (0,0,255) ,2, cv2.LINE_AA)    
        # === Extraindo ROI ===
        roi = cinza[y:y + h, x:x + w]      # extraindo ROI 
        roi = cv2.resize(roi, (48, 48))    # redimensiona para o tamanho aceito pelo modelo treinado
        roi = roi.astype("float") / 255.0  # normalizando
        roi = img_to_array(roi)            # convertendo para array para que a rede possa processar
        roi = np.expand_dims(roi, axis=0)  # mudando para o shape aceito pela cnn (com canal numerodeimagem, lagura, altura, canal)
    
    # === Aplicando Classificador ===
    # faz a predição - calcula as probabilidades
    result = model.predict(roi)[0]
    if result is not None:
        for (index, (emotion, prob)) in enumerate(zip(expressoes, result)):
        # nomes das emoções
            text = "{}: {:.2f}%".format(emotion, prob * 100)
            barra = int(prob * 150)  # calcula do tamanho da barra, com base na probabilidade
            espaco_esquerda = 7      # é a coordenada x onde inicia a barra. define quantos pixels tem de espaçamento à esquerda das barras, pra não ficar muito no canto. 
            if barra <= espaco_esquerda:
                barra = espaco_esquerda + 1
            # se o tamanho da barra for menor que o espaço da esquerda então deixa a barra com 1 pixel de largura
            # isso é feito pois estamos usando esse valor para passar por coordenada, se for menor então a barra irá crescer pra esquerda 
            cv2.rectangle(frame, (espaco_esquerda, (index * 18) + 7), (barra, (index * 18) + 18), (200, 250, 20), -1)
            cv2.putText(frame, text, (15, (index * 18) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 0, 0), 1, cv2.LINE_AA)

        resultado = np.argmax(result) # encontra a emoção com maior probabilidade
              
        cv2.putText(frame,'Estado: '+expressoes[resultado],(x,y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 1.1,(255,255,255),1,cv2.LINE_AA) # escreve a emoção acima do rosto
    
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) == ord('q'):
        break
captura.release()
cv2.destroyAllWindows()

In [11]:
x,y,w,h = faces[0]
